In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D,LSTM,Dense
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [165]:
#thousands allows to convert number with ',' to float
data = pd.read_csv("data/critics.notes.csv",sep=';;;; ') 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [166]:
data.loc[data["Note"] == 1]["Comment"]

0       film plutôt décevant! le rôle confié à Vincent...
14      Totalement inutile......c’est la seule femme q...
23      Huis clos au scénario anorexique. Sound of my ...
58      Je suis déçue, je m'attendais à mieux : le scé...
63      Adaptation du roman de Kerouac horriblement lo...
89      Un film sympa pour perdre sont temps, un film ...
142     Pour moi un film malsain ...beaucoup trop de s...
159     onirique.... aritistique... initiatique... fan...
165     mise a pars les effets speciaux spectaculurair...
166     'est un grand navet mais je lui met une étoile...
175     Une étoile pour le merveilleux jeu de Géraldin...
217     Grande déception !! Moi qui était une fan des ...
253     Bon scénario mais réalisation indigne. Florent...
273     Enième redite sur un thème déjà mille fois tra...
293     Ce Film est juste regardable, c'est au niveau ...
294                              Scenario bidon,a eviter!
323     je suis assez déçue par le film. je m attendai...
334     Je doi

In [168]:
data["Comment"] = data["Comment"].apply(lambda x: x.lower())

In [169]:
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split = ' ')
tokenizer.fit_on_texts(data["Comment"].values)
X = tokenizer.texts_to_sequences(data['Comment'].values)
X = pad_sequences(X)

### LSTM Model

In [170]:
embed_dim = 128
lstm_out = 196
num_classes = len(data['Note'].unique())

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1736, 128)         1280000   
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 1736, 128)         0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1970      
Total params: 1,536,770
Trainable params: 1,536,770
Non-trainable params: 0
_________________________________________________________________


In [171]:
data['Note'] = pd.to_numeric(data["Note"])

In [172]:
Y = pd.get_dummies(data['Note']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
#y_train_cat = to_categorical(Y_train, num_classes)
#y_test_cat = to_categorical(Y_test, num_classes)

In [173]:
#print(y_train_cat)

In [174]:
batch_size = 8
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
#model.fit(X_train, y_train_cat, epochs = 2, batch_size=batch_size, verbose = 2)

Epoch 1/20
 - 2057s - loss: 1.7522 - acc: 0.4632
Epoch 2/20
 - 1985s - loss: 1.5672 - acc: 0.4803
Epoch 3/20
 - 1990s - loss: 1.3249 - acc: 0.5473
Epoch 4/20
 - 1984s - loss: 1.0957 - acc: 0.6216
Epoch 5/20
 - 1982s - loss: 0.8703 - acc: 0.7145
Epoch 6/20
 - 1984s - loss: 0.6762 - acc: 0.7738
Epoch 7/20
 - 1982s - loss: 0.5428 - acc: 0.8264
Epoch 8/20
 - 1983s - loss: 0.4315 - acc: 0.8592
Epoch 9/20
 - 1982s - loss: 0.3385 - acc: 0.8922
Epoch 10/20
 - 1979s - loss: 0.2714 - acc: 0.9111
Epoch 11/20
 - 1981s - loss: 0.2139 - acc: 0.9289
Epoch 12/20
 - 1980s - loss: 0.1758 - acc: 0.9465
Epoch 13/20
 - 1983s - loss: 0.1463 - acc: 0.9494
Epoch 14/20
 - 1984s - loss: 0.1207 - acc: 0.9608
Epoch 15/20
 - 1989s - loss: 0.1065 - acc: 0.9670
Epoch 16/20
 - 1978s - loss: 0.0966 - acc: 0.9699
Epoch 17/20
 - 1979s - loss: 0.0844 - acc: 0.9720
Epoch 18/20
 - 1983s - loss: 0.0706 - acc: 0.9788
Epoch 19/20
 - 1986s - loss: 0.0591 - acc: 0.9811
Epoch 20/20
 - 1989s - loss: 0.0626 - acc: 0.9827


Extracting a validation set, and measuring score and accuracy.

In [175]:
validation_size = 100

#X_validate = X_test[-validation_size:]
#Y_validate = Y_test[-validation_size:]
#X_test = X_test[:-validation_size]
#Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
#score,acc = model.evaluate(X_test, y_test_cat, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 4.39
acc: 0.37


In [132]:
Y

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [179]:
#twt = ["Un chef d'oeuvre concluant magnifiquement l'épopée entamée en 2008 avec Iron Man. Le film de l'année sans difficulté. Frissons, émotions et scènes de combats épiques : on trouve vraiment tout ce qu'on attendait de ce film et plus encore."]
#twt = ["Enfin la boucle est bouclé, ce film est une excellente conclusion à 11 ans de Marvel Cinematic Universe, il alterne entre humour et moments d’émotions. La bataille finale est épique. On aurait presque envie de lâcher une larmichette à certains moments. Sûrement le meilleur Avengers des 4 films."]
#twt = ["Il y a du bon, du très bon, mais aussi du mauvais. Dans le bon, l'humour, le scénario (en particulier la seconde heure). Dans le mauvais la première heure assez longue et lente et surtout les incohérences classiques de chez Marvel, à savoir le super fort qui prend une déculottée ou le super faible qui écrase tout sur son passage... Mais dans la globalité l'histoire se laisse bien regarder, on rit assez souvent et les effets spéciaux sont réussis."]
#twt = ["Franchement j'ai failli ne pas aller voir ce film car la durée de 3 heures, c'est beaucoup trop long pour moi. Au bout des deux heures j'en avais déjà bien marre. Bon alors oui c'est un nouvel Avengers avec toute la clique. Je ne suis pas très fan mais j'ai quand même tenu à le voir. Il y a des passages supers et d'autres franchement pénibles. Le souci c'est comme je ne suis pas friande de ce genre de films, forcément je ne peux pas mettre la note maximale."]
#twt = ["Et bien moi je me suis un peu ennuyer et pour faire court il y a trop de retournement de cerveaux et je pense qu'ils ce sont fait bouffés par leurs scénarios Thor et hulk sont devenu figurant et pas de Come back prévu il fallait faire plus simple comme infinity war qui celui là était le meilleur de tous. Comme star War il y a beaucoup de potentiel gâché. Bien à vous"]
#twt = ["Trop long et scenario pas credible et frustrant de pouvoir revenir en arriere et dans le temps , ce qui annule tout interet au precedent episode. Dommage , car pour le reste ce n'etait pas trop mal avec une touche d'humour bienvenue."]
#twt = ["Déception pour le dernier film des frères Cohen. Llewyn Davis, musicien de son état, tente de vivre de la musique, sans rencontrer le succès escompté. Nous suivons ses pérégrinations dans le New-York des années 60. Sauf que le rythme du film est lent, et ce qui arrive à notre héros, ou plutôt anti-héros, n'est pas intéressant. Je me suis quelque peu ennuyé durant ce film. Cependant, la bande-son est magnifique et les airs folks chantés par Oscar Isaac sont superbement interprétés, pleins d'émotions. L'image est aussi d'une beauté rare, avec un espèce de voile blanc qui donne un esthétisme indéniable au film. Mais pour moi, cela n'occulte pas les longueurs du film ainsi que son scénario n'ayant que peu d'intérêt"]
twt = ["Film très très décevant. On s'attend à un REC dans la lignée des précédents et ça n'a rien à avoir. On se retrouve avec un film gore, violent sans tension. On ne ressent pas la peur ou l'oppression du premier. Ca n'a rien à voir en fait. Ca aurait pu être un bon film s'il ne s'appelait pas RECMais là, ce comico burlesque gore tombe un peu à plat. La mariée qui se transforme en super héroine sans coeur en quelques minutes ,c'est assez gros quand même.Dans le genre Shaun of the Dead est quand même bien meilleur"]

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=X.shape[1], dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
#print(sorted(data['Note'].unique()))
print(data['Note'].unique())
predicted_label = sorted(data['Note'].unique())[sentiment.argmax(axis=-1)]
#predicted_label = data['Note'].unique()[sentiment.argmax(axis=-1)]
print(predicted_label)

[ 1.   4.5  2.   5.   4.   3.   2.5  3.5  1.5  0.5]
3.0


In [154]:
data['Note'].unique()

array([ 5. ,  4. ,  1. ,  2. ,  1.5,  4.5,  2.5,  3. ,  3.5,  0.5])